<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-19 12:54:24
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: 9.39 K (0.06%)
Current PnL: -20.89 L (-13.69%)
CY Booked + Current PnL: -6.93 L (-4.54%)
-------------------
Total profit:  1.89 L
Total loss:  -22.78 L
-------------------
Total Booked + Current PnL: 20.05 L (15.96%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.63%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 90.24 L (62.25%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.91,-7.61,8.25,0.02,13306.0,-13278.0,161281.0,146.09,78.0,M-SC,1.87,234.0,-1.00,1.11,40.29,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,0.09,-14.14,16.58,0.09,14346.0,-14251.0,86526.0,100.85,50.0,M-SC,3.72,253.0,-0.99,0.60,13.68,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-0.10,-1.96,18.76,16.44,16679.0,-1773.0,88905.0,-14.93,49.0,X-MC,3.84,63.0,-0.11,0.61,17.83,X40,ATH,PHARMA
35,ICICIGI,2252.93,1.08,9.24,10.02,20.19,19082.0,16116.0,190436.0,-14.37,59.0,X-MC,7.61,68.0,0.84,1.31,25.76,X40,ATH,INSURANCE
84,VOLTAS,1530.00,1.05,9.43,9.38,19.69,19681.0,18078.0,209820.0,-1.22,56.0,X-MC,3.72,78.0,0.92,1.45,16.87,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7201.88,4.34,9.80,19.91,31.66,53811.0,24119.0,270270.0,5.72,68.0,H-LC,11.48,43.0,0.45,1.86,49.96,X200,ATH,IT
32,HCLTECH,1922.01,4.15,7.82,15.69,24.73,40926.0,18916.0,260840.0,12.76,77.0,X-LC,4.38,7.0,0.46,1.80,24.25,X40,ATH,IT
13,BSOFT,831.70,3.40,-28.80,111.76,50.79,111929.0,-40502.0,100151.0,-8.77,62.0,H-SC,8.21,151.0,-0.36,0.69,16.61,XR,ATH,IT
69,SONACOMS,806.63,3.21,-12.77,59.97,39.54,52920.0,-12915.0,88244.0,-31.83,69.0,M-SC,6.77,220.0,-0.24,0.61,24.43,AR,ATH,AUTO
41,INFY,2275.00,3.20,4.43,48.31,54.87,157849.0,13854.0,326742.0,-18.52,58.0,X-LC,3.63,2.0,0.09,2.25,13.08,X40,BTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.0,-3.56,-36.35,77.39,12.90,107625.0,-79427.0,139069.0,104.33,60.0,H-SC,16.03,161.0,-0.74,0.96,15.63,OX40N,NTT,FINANCE
64,SATIN,274.0,-2.24,-15.77,80.26,51.83,186775.0,-43572.0,232712.0,-25.10,52.0,H-SC,2.50,146.0,-0.23,1.61,13.37,XY24,NTT,FINANCE
77,TMPV,1065.0,-1.93,-13.71,192.46,152.37,459052.0,-37892.0,238518.0,-23.08,22.0,X-LC,0.26,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
63,SAMMAANCAP,326.0,-1.92,8.11,81.87,96.62,132077.0,12105.0,161325.0,19.75,54.0,M-SC,36.62,245.0,0.09,1.11,74.59,XY25,NTT,FINANCE
39,INDIGOPNTS,1408.0,-1.91,-7.61,8.25,0.02,13306.0,-13278.0,161281.0,146.09,78.0,M-SC,1.87,234.0,-1.00,1.11,40.29,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.0,-0.12,-0.35,47.78,47.26,133848.0,-990.0,280133.0,36.05,66.0,H-SC,15.6,171.0,-0.01,1.93,50.09,XY24,NTT,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.73,-2.27,112.75,107.91,162686.0,-3355.0,144289.0,-20.55,41.0,M-SC,10.14,216.0,-0.02,1.00,0.66,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.91,-7.61,8.25,0.02,13306.0,-13278.0,161281.0,146.09,78.0,M-SC,1.87,234.0,-1.00,1.11,40.29,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,0.09,-14.14,16.58,0.09,14346.0,-14251.0,86526.0,100.85,50.0,M-SC,3.72,253.0,-0.99,0.60,13.68,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.77,-18.21,38.74,13.47,85443.0,-49113.0,220554.0,-66.37,43.0,H-SC,1.99,158.0,-0.57,1.52,12.15,XY24,NTT,PAINTS
74,TATAELXSI,9161.00,1.88,-22.52,70.41,32.03,71920.0,-29691.0,102144.0,-16.20,50.0,H-SC,6.33,148.0,-0.41,0.70,14.41,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-1.92,8.11,81.87,96.62,132077.0,12105.0,161325.0,19.75,54.0,M-SC,36.62,245.0,0.09,1.11,74.59,XY25,NTT,FINANCE
86,WIPRO,420.00,2.07,1.00,70.81,72.51,107951.0,1507.0,152452.0,-13.34,55.0,M-LC,5.13,99.0,0.01,1.05,7.08,XR,NTT,IT
38,INDIAMART,4810.62,-1.51,3.53,99.67,106.72,127272.0,4357.0,127693.0,-48.72,44.0,H-SC,4.56,139.0,0.03,0.88,27.74,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.51,3.53,99.67,106.72,127272.0,4357.0,127693.0,-48.72,44.0,H-SC,4.56,139.0,0.03,0.88,27.74,AR,ATH,MISC
86,WIPRO,420.00,2.07,1.00,70.81,72.51,107951.0,1507.0,152452.0,-13.34,55.0,M-LC,5.13,99.0,0.01,1.05,7.08,XR,NTT,IT
25,FINCABLES,1641.55,-1.73,-2.27,112.75,107.91,162686.0,-3355.0,144289.0,-20.55,41.0,M-SC,10.14,216.0,-0.02,1.00,0.66,OX40N,ATH,CABLES
87,ZYDUSLIFE,1286.17,0.51,-1.60,37.84,35.63,78032.0,-3359.0,206215.0,-15.41,35.0,H-MC,5.24,119.0,-0.04,1.42,14.80,AR,ATH,PHARMA
85,WHIRLPOOL,2270.00,-0.06,-3.33,88.19,81.92,97864.0,-3828.0,110970.0,-43.39,35.0,M-SC,7.83,236.0,-0.04,0.77,31.37,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-1.93,-13.71,192.46,152.37,459052.0,-37892.0,238518.0,-23.08,22.0,X-LC,0.26,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.45,-34.84,107.48,35.20,90119.0,-44823.0,83847.0,9.89,24.0,X-SC,10.90,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.09,-18.40,67.69,36.84,70668.0,-23538.0,104400.0,-26.58,28.0,X-SC,1.31,90.0,-0.33,0.72,6.88,X40N,ATH,ELECTRICAL
26,GILLETTE,11206.78,-0.41,-1.82,34.87,32.42,86926.0,-4611.0,249285.0,-19.22,28.0,X-SC,6.11,85.0,-0.05,1.72,13.01,X40,ATH,FMCG
56,QUESS,424.00,-1.26,-29.90,103.76,42.84,47286.0,-19434.0,45572.0,-54.27,30.0,X-SC,37.29,83.0,-0.41,0.31,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-1.93,-13.71,192.46,152.37,459052.0,-37892.0,238518.0,-23.08,22.0,X-LC,0.26,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.00,-0.60,-1.44,12.05,10.44,23770.0,-2875.0,197263.0,-41.15,41.0,X-LC,0.92,5.0,-0.12,1.36,4.05,X40,NTT,FMCG
34,HONAUT,58357.33,-0.09,-18.40,67.69,36.84,70668.0,-23538.0,104400.0,-26.58,28.0,X-SC,1.31,90.0,-0.33,0.72,6.88,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-1.24,2.62,27.63,30.97,50112.0,4629.0,181367.0,-21.49,55.0,X-MC,1.45,75.0,0.09,1.25,15.51,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.58,-13.19,44.59,25.51,72031.0,-24555.0,161540.0,-0.76,62.0,H-LC,1.78,49.0,-0.34,1.11,19.04,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.45,-34.84,107.48,35.20,90119.0,-44823.0,83847.0,9.89,24.0,X-SC,10.90,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
56,QUESS,424.00,-1.26,-29.90,103.76,42.84,47286.0,-19434.0,45572.0,-54.27,30.0,X-SC,37.29,83.0,-0.41,0.31,0.00,XY24,NTT,MISC
54,PAGEIND,51605.07,-0.69,-5.07,32.80,26.07,50984.0,-8300.0,155440.0,-30.06,33.0,X-MC,8.19,55.0,-0.16,1.07,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-1.93,-13.71,192.46,152.37,459052.0,-37892.0,238518.0,-23.08,22.0,X-LC,0.26,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.45,-13.22,71.07,48.45,123665.0,-26502.0,174004.0,-19.44,30.0,X-MC,6.93,64.0,-0.21,1.20,0.34,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.26,-29.90,103.76,42.84,47286.0,-19434.0,45572.0,-54.27,30.0,X-SC,37.29,83.0,-0.41,0.31,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.77,-45.84,181.24,52.31,142476.0,-66548.0,78612.0,-42.31,35.0,X-SC,3.10,91.0,-0.47,0.54,5.23,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.45,-34.84,107.48,35.20,90119.0,-44823.0,83847.0,9.89,24.0,X-SC,10.90,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-0.10,-1.96,18.76,16.44,16679.0,-1773.0,88905.0,-14.93,49.0,X-MC,3.84,63.0,-0.11,0.61,17.83,X40,ATH,PHARMA
34,HONAUT,58357.33,-0.09,-18.40,67.69,36.84,70668.0,-23538.0,104400.0,-26.58,28.0,X-SC,1.31,90.0,-0.33,0.72,6.88,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,1.71,-14.68,39.81,19.28,117503.0,-50796.0,295160.0,-25.57,62.0,X-LC,8.12,1.0,-0.43,2.04,9.12,X40,ATH,IT
41,INFY,2275.00,3.20,4.43,48.31,54.87,157849.0,13854.0,326742.0,-18.52,58.0,X-LC,3.63,2.0,0.09,2.25,13.08,X40,BTT,IT
77,TMPV,1065.00,-1.93,-13.71,192.46,152.37,459052.0,-37892.0,238518.0,-23.08,22.0,X-LC,0.26,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
83,VBL,671.64,-0.82,-7.50,46.68,35.67,136371.0,-23702.0,292140.0,-18.42,47.0,X-LC,6.03,4.0,-0.17,2.02,5.31,X40N,ATH,FMCG
43,ITC,452.00,-0.60,-1.44,12.05,10.44,23770.0,-2875.0,197263.0,-41.15,41.0,X-LC,0.92,5.0,-0.12,1.36,4.05,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.70,-10.10,102.09,81.67,86094.0,-9479.0,84331.0,7583.33,61.0,L-SC,18.02,271.0,-0.11,0.58,63.31,XR,NTT,CERAMICS
69,SONACOMS,806.63,3.21,-12.77,59.97,39.54,52920.0,-12915.0,88244.0,-31.83,69.0,M-SC,6.77,220.0,-0.24,0.61,24.43,AR,ATH,AUTO
51,MASFIN,398.61,-1.67,-4.47,27.76,22.05,25983.0,-4380.0,93600.0,-17.80,53.0,H-SC,7.19,164.0,-0.17,0.65,36.05,XR,ATH,FINANCE
71,SURYODAY,216.00,0.28,-21.10,55.93,23.03,79023.0,-37781.0,141290.0,53.51,45.0,H-SC,12.64,167.0,-0.48,0.97,40.19,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,-1.81,-22.35,107.03,60.77,151909.0,-40844.0,141931.0,63.64,49.0,H-SC,10.76,159.0,-0.27,0.98,27.82,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.91,-7.61,8.25,0.02,13306.0,-13278.0,161281.0,146.09,78.0,M-SC,1.87,234.0,-1.00,1.11,40.29,OX40N,NTT,PAINTS
13,BSOFT,831.70,3.40,-28.80,111.76,50.79,111929.0,-40502.0,100151.0,-8.77,62.0,H-SC,8.21,151.0,-0.36,0.69,16.61,XR,ATH,IT
5,ASIANPAINT,3460.25,0.11,1.68,18.94,20.95,48488.0,4242.0,256010.0,0.63,84.0,X-LC,10.88,31.0,0.09,1.77,37.92,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,-1.81,-22.35,107.03,60.77,151909.0,-40844.0,141931.0,63.64,49.0,H-SC,10.76,159.0,-0.27,0.98,27.82,XR,NTT,JEWELLERY
32,HCLTECH,1922.01,4.15,7.82,15.69,24.73,40926.0,18916.0,260840.0,12.76,77.0,X-LC,4.38,7.0,0.46,1.80,24.25,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.67
1,25,44.48
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,46.52
MC,28.65
LC,24.81


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,27.90
X40,20.60
X40N,11.52
XY25,10.12
XR,9.48
AR,9.04
OX40N,7.64
X200,1.86
SR,1.01


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,24.21
X-MC,22.27
X-LC,19.82
M-SC,13.21
X-SC,7.70
H-MC,4.69
H-LC,2.97
L-SC,1.40
M-MC,1.36


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.60,-4.61,38.45
IT,13.59,-12.98,71.36
FINANCE,10.46,-12.12,64.85
MISC,7.36,-21.60,71.41
PAINTS,6.05,-5.30,20.94
ELECTRICAL,5.62,-10.92,50.25
INSURANCE,4.49,3.03,31.14
PHARMA,3.42,-0.65,34.42
AUTO,3.34,-18.78,132.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3278185.0,21
AR,1259440.0,10
XR,1256967.0,13
X40,982087.0,14
X40N,701726.0,9
OX40N,683607.0,10
XY25,465853.0,7
SR,275376.0,2
MH,66573.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3468579.0,25
M-SC,1438433.0,16
X-LC,1194817.0,11
X-MC,1166314.0,16
X-SC,724450.0,8
H-MC,399149.0,3
L-SC,257140.0,3
H-LC,125842.0,2
M-LC,107951.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1178426.0      6
           AR         850152.0      5
           XR         778953.0      7
M-SC       XY24       761770.0      6
X-LC       XY24       529394.0      2
X-MC       X40        416506.0      7
X-LC       X40        388536.0      5
X-MC       XY24       344159.0      3
H-SC       OX40N      319099.0      4
X-SC       XY24       280793.0      3
M-SC       OX40N      277003.0      5
H-SC       SR         275376.0      2
X-SC       X40N       266612.0      3
X-MC       X40N       230767.0      4
H-MC       AR         215506.0      2
X-LC       X40N       204347.0      2
H-MC       XY24       183643.0      1
X-SC       X40        177045.0      2
X-MC       XY25       174882.0      2
L-SC       XR         169635.0      2
M-SC       XR         145832.0      2
           XY25       132077.0      1
           AR         121751.0      2
M-LC       XR         107951.0      1
L-SC       OX40N       87505.0      1
X-LC       XY25        72540.0      2
H-LC       AR          72031.0      1
H-SC       MH          66573.0      1
L-MC       XR          54596.0      1
H-LC       X200        53811.0      1
M-MC       XY25        47829.0      1
L-LC       XY25        38525.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
